# Natural language processing introductory challenge
This challenge was built to introduce someone to applying machine learning to problems of natural language processing. In particular, it aims at detecting natural disasters from tweets.